In [1]:
pip install google-generativeai

In [4]:
import google.generativeai as genai

def set_gemini_params(
    model="gemini-pro",
    temperature=0.7,
    max_output_tokens=256,
    top_p=1,
    top_k=40,
):
    """Set Gemini API parameters"""

    gemini_params = {
        "model": model,
        "temperature": temperature,
        "max_output_tokens": max_output_tokens,
        "top_p": top_p,
        "top_k": top_k,
    }
    return gemini_params

def get_gemini_completion(params, prompt):
    """Get completion from Gemini API"""

    genai.configure(api_key="")
    model = genai.GenerativeModel(params["model"])

    response = model.generate_content(
        prompt,
        generation_config={
            "temperature": params["temperature"],
            "max_output_tokens": params["max_output_tokens"],
            "top_p": params["top_p"],
            "top_k": params["top_k"],
        }
    )
    return response.text  # Return generated text

params = set_gemini_params()

In [12]:
zero_shot_prompt = "Write a Python function to find the second-largest number in a given list of integers."

response = get_gemini_completion(params, zero_shot_prompt)

print("Zero-Shot Learning Output:\n", response)


Zero-Shot Learning Output:
 ```python
def second_largest(nums):
  """
  Finds the second-largest number in a given list of integers.

  Parameters:
    nums: A list of integers.

  Returns:
    The second-largest number in the list, or None if the list is empty.
  """

  if not nums:
    return None

  largest = nums[0]
  second_largest = None

  for num in nums:
    if num > largest:
      second_largest = largest
      largest = num
    elif num < largest and (second_largest is None or num > second_largest):
      second_largest = num

  return second_largest
```


In [13]:
few_shot_prompt = """Write a Python function to find the second-largest number in a list of integers.

Example 1:
Input: [10, 20, 5, 8]
Output: 10

Example 2:
Input: [3, 3, 2, 1]
Output: 2

Example 3:
Input: [7]
Output: None (List must have at least two distinct numbers)

Your function should handle cases where all numbers are the same or where the list is too short."""

params = set_gemini_params(temperature=0.9)
response = get_gemini_completion(params, few_shot_prompt)

print("Few-Shot Learning Output:\n", response)


Few-Shot Learning Output:
 ```python
def second_largest(nums):
  """Finds the second-largest number in a list of integers.

  Args:
    nums: A list of integers.

  Returns:
    The second-largest number in the list, or None if the list is empty or
    has only one distinct number.
  """

  if len(nums) < 2:
    return None

  largest = max(nums)
  second_largest = None

  for num in nums:
    if num != largest and (second_largest is None or num > second_largest):
      second_largest = num

  return second_largest
```


In [16]:
cot_prompt = "Write a Python function to find the Find the second-largest number in a list of integers. Let's think step by step"

response = get_gemini_completion(params, cot_prompt)

print("Zero-Shot Chain of Thought Output:\n", response)


Zero-Shot Chain of Thought Output:
 ```python
def second_largest(lst):
    """
    Find the second-largest number in a list of integers.

    Parameters:
    lst: A list of integers.

    Returns:
    The second-largest number in the list, or None if the list is empty.
    """

    # Check if the list is empty.

    if not lst:
        return None

    # Find the largest number in the list.

    largest = lst[0]
    for number in lst:
        if number > largest:
            largest = number

    # Find the second-largest number in the list.

    second_largest = None
    for number in lst:
        if number != largest and (second_largest is None or number > second_largest):
            second_largest = number

    return second_largest
```


In [17]:
cot_custom_prompt =prompt = """Write a Python function to find the second-largest number in a given list.

Follow these steps:
1. Identify the largest number.
2. Ignore duplicates and find the second-largest number.
3. Return None if no second-largest exists.

Example 1:
Input: [12, 35, 1, 10, 34, 1]
Step 1: Largest number is 35.
Step 2: Second-largest is 34.
Output: 34

Example 2:
Input: [10, 10, 10]
Step 1: Largest number is 10.
Step 2: No second-largest exists.
Output: None

Now, write the function in Python.
"""

response = get_gemini_completion(params, cot_custom_prompt)

print("Chain of Thought Output:\n", response)

Chain of Thought Output:
 ```python
def find_second_largest(nums):
  """
  Finds the second-largest number in a given list.

  Parameters:
    nums: A list of numbers.

  Returns:
    The second-largest number in nums, or None if no second-largest
    number exists.
  """

  # Find the largest number.
  largest = max(nums)

  # Ignore duplicates and find the second-largest number.
  second_largest = None
  for num in nums:
    if num != largest and (second_largest is None or num > second_largest):
      second_largest = num

  # Return None if no second-largest exists.
  return second_largest
```


**Findings:**



1. Zero-Shot is unreliable for complex coding problems

    * Often generates incomplete or incorrect code.
    * Doesn't handle edge cases well.

2. Few-Shot Learning significantly improves accuracy

    * Examples guide the model to produce structured and correct code.
    * Works well for standard problems but may still fail in complex logic.

3. Zero-Shot Chain of Thought (CoT) improves reasoning

    * When prompted to explain step-by-step, the model makes fewer mistakes.
    * Still less effective than Few-Shot + CoT.

4. Few-Shot + Chain of Thought (Best Performance)

    * Combining examples + step-by-step reasoning produces the most accurate and robust code.
    * Handles edge cases better and provides explanations.




